In [1]:
from ddpg.DDPG import DDPG
from ddpg.portfolio import PortfolioEnv
from ddpg.StockActor import StockActor
from ddpg.StockCritic import StockCritic
from ddpg.ornstein_uhlenbeck import OrnsteinUhlenbeckActionNoise

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import json 

import os
fileDir = os.getcwd()

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
with open(fileDir+'/config/cnn.json', 'r') as f:
    config = json.load(f)

dataset_path = fileDir + '/data/data1130'
window_size = config["window_length"]
steps = config["steps"]
trading_cost = config["trading_cost"]
start_idx = config["start_idx"]

In [ ]:
env = PortfolioEnv(dataset_path,
                   window_size=window_size,
                   steps=steps,
                   trading_cost=trading_cost,
                   start_idx = start_idx)

info = env.reset()
asset_dim = info["observation"].shape[0]
feature_dim = info["observation"].shape[2]

In [ ]:
actor_learning_rate = config["actor_learning_rate"]
critic_learning_rate = config["critic_learning_rate"]
tau = config["tau"]
gamma = config["gamma"]
batch_size = config["batch_size"]
nn = config["nn"]
path = config["path"]

In [ ]:
with tf.Session() as sess:
    actor = StockActor(sess, asset_dim, window_size, feature_dim, actor_learning_rate, tau, batch_size, nn)
    num_actor_vars = actor.num_trainable_vars
    critic = StockCritic(sess, asset_dim, window_size, feature_dim, critic_learning_rate, tau, gamma, num_actor_vars, nn)
    actor_noise = OrnsteinUhlenbeckActionNoise(mu=np.zeros(asset_dim))

    ddpg = DDPG(env, sess, actor, critic, actor_noise, config, nn, 
                model_save_path = fileDir+ path + "ckpt",
                summary_path = fileDir + path + "logdir", 
                train_info_path = fileDir + path + "train_info_cnn.csv")
    ddpg.train()
ddpg.env.save_info(info_path = fileDir + path + "info_cnn.csv")